In [3]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

## Key Word Summarization

In [4]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en_core_web_sm')

# Parsing Gatsby.
gatsby = parser(gatsby)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [5]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])

(0.0639701861589625, 'The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men.')


In [6]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [7]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

C:\Users\rabia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[(0.012415518587086127, promises), (0.012415518587086127, exempt), (0.012333160533862523, glimpses), (0.012082343425981395, intimate), (0.012054574145630725, earthquakes)]


## Digrams Key Word Summrization

In [49]:
# creation of digrams

gatsby2 = []

for i in range(len(gatsby)):
    digram = gatsby[i:i+2]
    gatsby2.append(digram)

In [95]:
gatsby2[:5]

[In my, my younger, younger and, and more, more vulnerable]

In [159]:
# remove punctuation from our digrams
gatsby2a = [digram for digram in gatsby2[:-2] if digram[0].is_punct==False and digram[1].is_punct==False 
                and digram[0].is_stop==False and digram[1].is_stop==False]

gatsby2_filt = [digram for digram in gatsby2a if digram[0].pos_=='ADJ' and digram[1].pos_=='NOUN'
               or digram[1].pos_=='ADJ' and digram[0].pos_=='NOUN']

digrams=set(gatsby2_filt)

adjacency2=pd.DataFrame(columns=[str(i) for i in digrams] ,index=[str(i) for i in digrams],data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,d in enumerate(gatsby2a):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([d == item for item in gatsby2_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby2a)-(len(gatsby2a)-(i+10)))
        # The potential neighbors.
        nextwords=gatsby2a[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby2_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            for i in neighbors:
                adjacency2.loc[str(d),str(i)]=adjacency2.loc[str(d),str(i)]+1

print('done!')

done!


In [160]:
# Running TextRank
nx_words2 = nx.from_numpy_matrix(adjacency2.as_matrix())
ranks2=nx.pagerank(nx_words2, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked2 = sorted(((ranks2[i],s) for i,s in enumerate(digrams)),
                reverse=True)
print(ranked2[:5])

C:\Users\rabia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[(0.034903513588847704, unknown men), (0.034903513588847704, secret griefs), (0.03471183685843079, hostile levity), (0.03465719424411963, successful gestures), (0.03398694207346373, unbroken series)]


## Trigrams Key Word Summrization

In [91]:
# creation of trigrams

gatsby3 = []

for i in range(len(gatsby)):
    trigram = gatsby[i:i+3]
    gatsby3.append(trigram)

In [96]:
gatsby3[:5]

[In my younger,
 my younger and,
 younger and more,
 and more vulnerable,
 more vulnerable years]

In [183]:
# remove punctuation from our digrams
gatsby3a = [trigram for trigram in gatsby3[:-2] if trigram[0].is_punct==False and trigram[1].is_punct==False 
            and trigram[2].is_punct==False
            and trigram[2].is_stop==False]
            

gatsby3_filt = [trigram for trigram in gatsby3a 
                if trigram[0].pos_=='NOUN' or trigram[1].pos_=='NOUN' and trigram[2].pos_=='NOUN']

trigrams=set(gatsby3_filt)

adjacency3=pd.DataFrame(columns=[str(i) for i in trigrams] ,index=[str(i) for i in trigrams],data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,t in enumerate(gatsby3a):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([d == item for item in gatsby3_filt]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby3a)-(len(gatsby3a)-(i+15)))
        # The potential neighbors.
        nextwords=gatsby3a[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby3_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            for i in neighbors:
                adjacency3.loc[str(t),str(i)]=adjacency3.loc[str(t),str(i)]+1

print('done!')

done!


In [185]:
# Running TextRank
nx_words3 = nx.from_numpy_matrix(adjacency3.as_matrix())
ranks3=nx.pagerank(nx_words3, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked3 = sorted(((ranks3[i],s) for i,s in enumerate(trigrams)),
                reverse=True)
print(ranked3[:5])

C:\Users\rabia\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[(0.04761904761904764, elations of men), (0.04761904761904764, sorrows and short), (0.04761904761904764, dreams that temporarily), (0.04761904761904764, gift for hope), (0.04761904761904764, earthquakes ten thousand)]
